<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/navigator-data-designer-sdk-structured-outputs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Data Designer SDK: Structured Outputs

Let's explore how to use Data Designer's structured outputs feature to generate complex, nested data structures, with support for both Pydantic and JSON schema definitions.

# 📘 Getting Started

First, let's install and import the required packages:

In [1]:
%%capture
# Install the latest version of Gretel client and dependencies
%pip install -U git+https://github.com/gretelai/gretel-python-client

In [8]:
from gretel_client.navigator_client import Gretel

# 🥗 Building a Fruit Salad Generator

To demonstrate structured outputs, we'll create a fruit salad recipe generator. This example showcases how to:
- Handle nested data structures (recipes containing multiple fruits)
- Generate variable-length lists (different numbers of fruits per salad)
- Maintain relationships between components (total cost based on individual fruits)
- Create derivative content (HTML presentations of our recipes)

## 1. Setting Up Data Designer

First, we'll create a Data Designer instance and define our seed data:

In [ ]:
## Create our DD Instance
gretel = Gretel(api_key="prompt", endpoint='https://api.dev.gretel.ai')

aidd = gretel.data_designer.new(
    model_suite="apache-2.0"  # Use apache-2.0 or llama-3.x based on your licensing needs
)

In [ ]:

## Generate some regions for our fruit salad recipes
aidd.add_column(
    name="region",
    type="category",
    params={"values": ["Thailand", "France", "South Africa"]},
)

## 2. Defining Our Data Model

The power of structured outputs comes from defining exact schemas for our generated data. We'll use Pydantic to create our data models:

In [11]:
## Now, we're making a recipe, which is pretty structured.
## So let's give data designer a recipe to follow!

from pydantic import BaseModel, Field

class Fruit(BaseModel):
    name: str = Field(..., description="Name of the fruit.")
    cost: float = Field(..., description="Dollar value of the fruit.")
    weight: float = Field(..., description="Weight in lbs.")
    flavor: str = Field(..., description="Primary flavor profile of the fruit.")
    preparation: str = Field(..., description="How to prepare the fruit for a fruit salad.")


class FruitSalad(BaseModel):
    total_cost: float = Field(..., description="Total cost of all fruits.")
    name: str = Field(..., description="Name of this unique fruit salad.")
    haiku: str = Field(..., description="A beautiful haiku about this fruit salad.")
    ascii_art: str = Field(..., description="A small ASCII art depiction of the fruit salad.")
    fruits: list[Fruit]

Now, we can pass ths Pydantic data model to DataDesigner and have a contract that we'll get back data in the format we specified above (or none!)

Our implementation also permits one to specify [JSON Schema](https://json-schema.org/) directly via any source.


## 3. Generating Structured Data

With our models defined, we can now tell Data Designer exactly what kind of data to generate:

In [ ]:
## Tell DD to generate some fruit salads
aidd.add_column(
    name="fruit_salad",
    type="llm-gen",
    prompt=(
        "Create a description of fruits to go in a regional fruit salad from {{region}}!"
    ),
    data_config={"type": "structured", "params": {"model": FruitSalad}}
)

## 4. Previewing Results

Let's take a look at what we've created:

In [ ]:
preview = aidd.preview()

In [ ]:
preview.display_sample_record()